# 加权Max-Sat示例


##问题语句
最大满意问题（MAX-SAT）是确定最大值的问题
符合性正常的布尔公式的条款的数量，因为它们可能是
威胁通过分配的方式将其价值变为形式的变量。
一个可以定义一个加权版本的MAX-SAT作为一个折叠：给定一个符合性正常
在某些条款中，以权重表示形式，并为其变量找到trut值。
它最大限度地提高了讽刺性条款的组合权重。
MAX-SAT问题是加权MAX-SAT的所有权重1。

##整数线性规划公式
MAX-SAT可以被挤出使用二进制线性程序。
修正了一个符合性正常形式的公式$Fs$$x_1，x_2，…，x_n$，
美元的价值。
对于$C$，$S$+$和$S$S$和$S$
它不会被否定为$c$，并且它被否定为$c$，
措辞的变量$y_x$将与措辞的变量$F$对应，
将变量$z_c$与索赔相关联。
如下：

maximize $\sum_{c \in C} w_c\cdot z_c$
(最大限度地提高讽刺性条款的重量)

subject to

$ z_c\leq\sum_{x\in S_c^+} y_x+\sum_{x\in S_c^-} (1-y_x)$
for all $c\in C$ (子句为真，当且仅当它有一个为真的非否定变量或一个为假的否定变量)

$z_c \in \{0,1\}$
for all $c\in C$
(每个条款要么满足，要么不满足。)

$y_x \in \{0,1\}$
for all $x\in F$
(每个变量不是真就是假。)

##粒子问题实例（加权MAX-SAT）

$\phi = ({x_1} \vee \overline{x_2} : 3) \wedge (x_3 : 1) \wedge (\overline{x_3} \vee x_2 : 4)) $

## 使用普通Python制定等效目标函数

In [6]:
def f(x):
    x1, x2, x3 = x
    val = 0

    # clause 1
    if x1 or not x2:
        val += 3
    # clause 2
    if x3:
        val += 1
    # clause 3
    if not x3 or x2:
        val += 4

    return val

## 在本地环境中安装软件包（需要colab）

In [7]:
!git clone https://github.com/FujitsuResearch/autoqubo
!cd autoqubo && python setup.py install
!cd autoqubo && pip install -r requirements.txt


fatal: destination path 'autoqubo' already exists and is not an empty directory.


running install
running bdist_egg
running egg_info
writing autoqubo.egg-info\PKG-INFO
writing dependency_links to autoqubo.egg-info\dependency_links.txt
writing requirements to autoqubo.egg-info\requires.txt
writing top-level names to autoqubo.egg-info\top_level.txt
reading manifest file 'autoqubo.egg-info\SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'autoqubo.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\autoqubo
copying build\lib\autoqubo\__init__.py -> build\bdist.win-amd64\egg\autoqubo
copying build\lib\autoqubo\_package_info.py -> build\bdist.win-amd64\egg\autoqubo
copying build\lib\autoqubo\binarization.py -> build\bdist.win-amd64\egg\autoqubo
copying build\lib\autoqubo\penalty_weights.py -> build\bdist.win-amd64\egg\autoqubo
copying build\lib\autoqubo\sampling_compiler.py -> build\bdist.win-amd64\egg\autoqubo
copying build\li

c:\Users\justi\miniconda3\envs\autoqubo\lib\site-packages\setuptools\command\install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
c:\Users\justi\miniconda3\envs\autoqubo\lib\site-packages\setuptools\command\easy_install.py:147: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
zip_safe flag not set; analyzing archive contents...
error: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\users\\justi\\miniconda3\\envs\\autoqubo\\lib\\site-packages\\autoqubo-0.0.1-py3.7.egg'


## 使用AutoQUBO转换为QUBO

In [8]:
# Import everything
from autoqubo import SamplingCompiler, Utils
from itertools import product

# Create QUBO using the sampling compiler
# The solution is represetned by 3 bits
qubo, offset = SamplingCompiler.generate_qubo_matrix(fitness_function=f, use_multiprocessing=False, input_size=3)

## 打印Qubo矩阵和系数

In [9]:
print("QUBO matrix:")
print(qubo)

print("QUBO coefficients")
print(f"x[] = {offset}")
for key, coefficient in Utils.get_matrix_dict_repr(qubo).items():
    i, j = key
    if i == j:
        print(f"x[{i+1}] = {coefficient}")
    else:
        print(f"x[{i+1}, {j+1}] = {coefficient}")


QUBO matrix:
[[ 0.  3.  0.]
 [ 0. -3.  4.]
 [ 0.  0. -3.]]
QUBO coefficients
x[] = 7
x[1, 2] = 3.0
x[2] = -3.0
x[2, 3] = 4.0
x[3] = -3.0


## 使用Qubo解算器最小化

In [10]:
solutions, energy_values = Utils.solve(qubo, offset)
for s, e in zip(solutions, energy_values):
    print(f"x={s}, energy={e}")

x=[1, 0, 1], energy=4.0
x=[0, 0, 1], energy=4.0
x=[0, 1, 0], energy=4.0
